In [5]:
import copy
# TODO: investigate death works well and internal death stats
# TODO: fix die-adaptive lr, show usefull stat instead of logs
# TODO: Collect internal statistics of history of Neurons at their death point (like: values, how fast changed, etc.)
# "batch norm analog" - i.e. collector neuron - when things will go slow with reasonable dataset - likely cifar10, then see if speed improvement
# Separate livenet core and high-level api?
# TODO: make amount of dangle/useless neurons updated during Neuron creation/connection?

import random
import numpy as np
import torch
import torch.nn as nn
import torch.utils.data
import importlib
from ai_libs.basic.deep_reload import deep_reload
import livenet
deep_reload(livenet)
core = livenet.core
import ai_libs.simple_log as simple_log
from ai_libs.simple_log import LOG
simple_log.level = simple_log.LogLevel.INFO
import math
from matplotlib import pyplot as plt
%matplotlib
plt.ion()
livenet.core.utils.set_seed()
var = 42
print(torch.__version__)
np.set_printoptions(precision=3)
%precision 4



Using matplotlib backend: module://matplotlib_inline.backend_inline
2.3.1+cu121


'%.4f'

In [8]:
downscale = (8, 8)
test_x, test_y = livenet.datasets.to_plain(*livenet.datasets.get_cifar10_test(), downscale=downscale, to_gray=True)
train_x, train_y = livenet.datasets.to_plain(*livenet.datasets.get_cifar10_train(), downscale=downscale, to_gray=True)


Files already downloaded and verified
Files already downloaded and verified


In [12]:
context = core.livenet.Context()
network = livenet.nets.create_perceptron(train_x.shape[1], 11, 10, context)
batch_iterator = livenet.gen_utils.batch_iterator(train_x, train_y, batch_size=1000)
criterion = livenet.nets.criterion_classification_n
optimizer = livenet.nets.create_optimizer(network)
optimizer.learning_rate = 0.01
trainer = livenet.net_trainer.NetTrainer(network, batch_iterator, criterion, optimizer, epoch_size=10)

LiveNet


In [23]:
network.context.regularization_l1 = 0.001
optimizer.learning_rate = 0.001
network.context.learning_rate = 0.001
trainer.step(1000)


Iˈ0.000 2.899 = 2.786+0.113reg params=178 livenet/net_trainer.py:93
Iˈ0.362 2.914 = 2.801+0.113reg params=178 livenet/net_trainer.py:93
Iˈ0.703 2.905 = 2.792+0.113reg params=178 livenet/net_trainer.py:93
Iˈ1.042 2.895 = 2.782+0.113reg params=178 livenet/net_trainer.py:93
Iˈ1.381 2.896 = 2.783+0.113reg params=178 livenet/net_trainer.py:93
Iˈ1.723 2.911 = 2.798+0.113reg params=178 livenet/net_trainer.py:93
Iˈ2.062 2.887 = 2.774+0.113reg params=178 livenet/net_trainer.py:93
Iˈ2.400 2.903 = 2.790+0.113reg params=178 livenet/net_trainer.py:93
Iˈ2.741 2.896 = 2.783+0.113reg params=178 livenet/net_trainer.py:93
Iˈ3.080 2.914 = 2.801+0.113reg params=178 livenet/net_trainer.py:93
Iˈ3.416 2.900 = 2.787+0.113reg params=178 livenet/net_trainer.py:93
Iˈ3.751 2.900 = 2.786+0.113reg params=178 livenet/net_trainer.py:93
Iˈ4.088 2.919 = 2.806+0.113reg params=178 livenet/net_trainer.py:93
Iˈ4.426 2.904 = 2.791+0.113reg params=178 livenet/net_trainer.py:93
Iˈ4.765 2.891 = 2.778+0.113reg params=178 livene

In [24]:
pred = network(test_x)
pred_bin = np.argmax(pred.detach().numpy(), axis=1, keepdims=True)
diff = test_y - pred_bin
f"accuracy {len(diff[diff == 0]) / len(diff)}"


'accuracy 0.3089'

In [25]:
import re
name = "numpy-1.21.6-cp37-cp37m-manylinux_2_12_x86_64.manylinux2010_x86_64.whl"
pattern = r"^from\s+\.\s+import\s+\w+\s+"
name = re.fullmatch(pattern, name)
re.match()


SyntaxError: unmatched ')' (1051535749.py, line 3)

In [56]:
import re
import os, sys
dirpath = "/home/spometun/table/w3"
files = os.listdir(dirpath)

# Regular expression to match a .whl file name and capture the package name
# regex = r'^([a-zA-Z0-9_\-]+)\-([a-zA-Z0-9_.!+]+)\-([a-zA-Z0-9_.!+]+)\-([a-zA-Z0-9_.!+]+)\-([a-zA-Z0-9_.!+]+)\.whl$'
#    regex = r'^([a-zA-Z0-9_]+)\-(?:[0-9]|[_\-\.]|py3|py2|none|any|linux|manylinux|abi|x86|cp3|cp37m|post)+.whl$'

def get_wheel_package_name(file_name: str):
    regex = r'^([a-zA-Z0-9_]+)\-.*\.whl$'
    match = re.match(regex, file_name)
    if match:
        package_name = match.group(1)
    else:
        package_name = None
    return package_name


file_name = "MarkupSafe-2.1.3-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl"
print(get_wheel_package_name(file_name))
# n = 0
# f = 0
# for file_name in files:
#     n += 1
#     package_name = get_wheel_package_name(file_name)
#     if package_name is None:
#         f += 1
#         print(file_name)
#         break
#     print(package_name)
#     
# print(n, f)


MarkupSafe


In [ ]:
%matplotlib
plt.ion()

def param_amount(param):
    val = len(param["params"])
    return val

def param_max(param):
    val = np.max(np.abs(param["params"]))
    return val

def param_picker1(param):
    try:
        val0 = param["params"]["S262->D1"].item()
    except KeyError:
        val0 = 0.
    return val0

def get_param_values(history, picker):
    values = []
    for entry in history:
        values.append(picker(entry))
    return values

plt.figure(figsize=(16, 9))
values = get_param_values(trainer.history, param_amount)
plt.grid()
#plt.plot(values)

accum = core.stat_utils.AccumStat()
accum.add_value(trainer.network.parameters())
accum.plot()


In [ ]:
for name, p in sorted(network.named_parameters(),key=lambda x: len(x[0]) + ord(x[0][0]) / 1000.):
    print(name, f"{p.item():.1f}")


In [9]:
s = set()
s.add(2)
s.add(1)
2 in s
s.remove(3)

KeyError: 3

In [ ]:
core.livenet.export_onnx(network, "/home/spometun/model.onnx")
